In [ ]:
!pip install evaluate
from google.colab import drive
drive.mount('/content/drive')
!git clone https://github.com/sinhajiya/hf_emotion_classifier.git
import os
import sys
project_root = os.path.abspath("/content/hf_emotion_classifier")
if project_root not in sys.path:
    sys.path.insert(0, project_root)
from hf_data import *
from metrics import summarize_evaluation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
Mounted at /content/drive
Cloning into 'hf_emotion_classifier'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 60 (delta 23), reused 19 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (60/60), 1.26 MiB | 2.58 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [ ]:
import os, shutil, time
from datetime import datetime
from typing import Tuple, List, Dict

import torch
import torch.nn as nn
import torch.ao.quantization as quant
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizerFast, DataCollatorWithPadding
from datasets import load_dataset
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

In [ ]:
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
ds = emotions()
train = ds['train']
val = ds['validation']
test = ds['test']
l = labels_and_ids()
print(l)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
token_data = ds.map(lambda x: tokenize_batch(x, tokenizer), batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

split/train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

split/validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

split/test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
MODEL_NAME = "bert-base-uncased"
MAX_LEN = 128
BATCH_SIZE = 16
DROPOUT = 0.3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def encode_labels(train_df, val_df, test_df):
    le = LabelEncoder()
    t = train_df.copy()
    v = val_df.copy()
    te = test_df.copy()

    t["label"]  = le.fit_transform(t["label"])
    v["label"]  = le.transform(v["label"])
    te["label"] = te["label"].map(lambda x: le.transform([x])[0])

    return t, v, te, le, len(le.classes_)


In [ ]:
class EmotionDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.texts = df["text"].tolist()
        self.labels = df["label"].tolist()
        self.max_len = max_len
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }


def make_cpu_test_loader(test_df, tokenizer):
    ds = EmotionDataset(test_df, tokenizer, 128)
    return DataLoader(ds, batch_size=128, shuffle=False, num_workers=0)

In [ ]:
class EmotionPredictor(nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        out = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        pooled = out.last_hidden_state[:, 0]
        logits = self.classifier(self.dropout(pooled))

        loss = None
        if labels is not None:
            loss = nn.CrossEntropyLoss()(logits, labels)
        return loss, logits


class EmotionPredictorQuant(nn.Module):

    def __init__(self, num_labels):
        super().__init__()
        self.num_labels = num_labels

        # Build backbone
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

        torch.backends.quantized.engine = "fbgemm"
        self.qconfig = quant.get_default_qat_qconfig("fbgemm")

        # do not quantize embedding
        self.bert.embeddings.qconfig = None

        quant.prepare_qat(self, inplace=True)

    def load_prepared_state(self, artifacts_dir, device="cpu"):
        sd = torch.load(os.path.join(artifacts_dir, "prepared_qat_state.pt"), map_location=device)
        self.load_state_dict(sd, strict=False)
        self.to(device)
        self.eval()
        print(f"[QAT] Loaded QAT model from {artifacts_dir} onto {device}")

    def forward(self, input_ids, attention_mask, labels=None):
        out = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        pooled = out.last_hidden_state[:, 0]
        logits = self.classifier(pooled)

        loss = None
        if labels is not None:
            loss = nn.CrossEntropyLoss()(logits, labels)
        return loss, logits


In [ ]:
def load_prepared_qat_for_inference(artifacts_dir, device="cpu"):
    meta = joblib.load(os.path.join(artifacts_dir, "meta.pkl"))
    tokenizer = BertTokenizerFast.from_pretrained(artifacts_dir)
    le = joblib.load(os.path.join(artifacts_dir, "label_encoder.pkl"))
    num_labels = meta["num_labels"]

    model = EmotionPredictorQuant(num_labels)
    model.load_prepared_state(artifacts_dir, device=device)
    return model, tokenizer, le


def evaluate(model, loader, device="cpu"):
    model.eval()
    tot_loss = 0
    all_preds, all_lbls = [], []

    with torch.no_grad():
        for batch in loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            loss, logits = model(batch["input_ids"], batch["attention_mask"], batch["labels"])
            tot_loss += loss.item()
            preds = logits.argmax(1).cpu().numpy()
            all_preds.extend(preds)
            all_lbls.extend(batch["labels"].cpu().numpy())

    acc = accuracy_score(all_lbls, all_preds)
    f1 = f1_score(all_lbls, all_preds, average="macro")
    return tot_loss / len(loader), acc, f1


def predict_texts(model, tokenizer, label_encoder, texts, device="cpu"):
    enc = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN,
        return_tensors="pt"
    )
    enc = {k: v.to(device) for k, v in enc.items()}

    with torch.no_grad():
        _, logits = model(enc["input_ids"], enc["attention_mask"])
    preds = logits.argmax(1).cpu().numpy()
    return label_encoder.inverse_transform(preds)

In [ ]:
def setup_tokenizer():
    return BertTokenizerFast.from_pretrained(MODEL_NAME)

def load_and_prep_dataset(ds=ds):

    return train.to_pandas(), ds["validation"].to_pandas(), ds["test"].to_pandas()

In [ ]:

def run_qat_training_and_save(artifacts_dir="/content/drive/MyDrive/ANLP_weights/qat",epochs_fp32=1, epochs_qat=1):

    tokenizer = setup_tokenizer()
    train_df, val_df, test_df = load_and_prep_dataset()
    train_df, val_df, test_df, label_encoder, num_labels = encode_labels(
        train_df, val_df, test_df
    )

    os.makedirs(artifacts_dir, exist_ok=True)
    joblib.dump(label_encoder, os.path.join(artifacts_dir, "label_encoder.pkl"))
    tokenizer.save_pretrained(artifacts_dir)

    train_loader = DataLoader(
        EmotionDataset(train_df, tokenizer, MAX_LEN),
        batch_size=BATCH_SIZE, shuffle=True
    )
    val_loader = DataLoader(
        EmotionDataset(val_df, tokenizer, MAX_LEN),
        batch_size=BATCH_SIZE, shuffle=False
    )

    device = DEVICE


    fp32_model = EmotionPredictor(num_labels).to(device)
    optimizer = torch.optim.AdamW(fp32_model.parameters(), lr=2e-5)

    for epoch in range(epochs_fp32):
        fp32_model.train()
        loop = tqdm(train_loader, desc=f"FP32 Epoch {epoch+1}")

        for batch in loop:
            batch = {k: v.to(device) for k, v in batch.items()}
            loss, _ = fp32_model(batch["input_ids"], batch["attention_mask"], batch["labels"])

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loop.set_postfix(loss=loss.item())

        val_loss, val_acc, val_f1 = evaluate(fp32_model, val_loader, device)
        print(f"[FP32] Val Loss={val_loss:.4f} | Acc={val_acc:.4f} | F1={val_f1:.4f}")

    torch.save(fp32_model.state_dict(), os.path.join(artifacts_dir, "fp32_state.pt"))
    qat_model = EmotionPredictorQuant(num_labels).to(device)

    qat_model.load_state_dict(fp32_model.state_dict(), strict=False)

    optimizer = torch.optim.AdamW(qat_model.parameters(), lr=1e-5)


    for epoch in range(epochs_qat):
        qat_model.train()
        loop = tqdm(train_loader, desc=f"QAT Epoch {epoch+1}")

        for batch in loop:
            batch = {k: v.to(device) for k, v in batch.items()}
            loss, _ = qat_model(batch["input_ids"], batch["attention_mask"], batch["labels"])

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loop.set_postfix(loss=loss.item())

        val_loss, val_acc, val_f1 = evaluate(qat_model, val_loader, device)
        print(f"[QAT] Val Loss={val_loss:.4f} | Acc={val_acc:.4f} | F1={val_f1:.4f}")


    torch.save(
        qat_model.state_dict(),
        os.path.join(artifacts_dir, "prepared_qat_state.pt")
    )

    joblib.dump(
        {"num_labels": num_labels},
        os.path.join(artifacts_dir, "meta.pkl")
    )

    print("\n QAT ARTIFACTS SAVED TO:", artifacts_dir)

    # Optional: Create ZIP for Google Drive upload
    zip_path = artifacts_dir + ".zip"
    shutil.make_archive(artifacts_dir, 'zip', artifacts_dir)
    print(" Created ZIP file:", zip_path)

    return artifacts_dir, zip_path

run_qat_training_and_save()


 TRAINING FP32 MODEL 



FP32 Epoch 1: 100%|██████████| 1000/1000 [05:55<00:00,  2.81it/s, loss=0.187]


[FP32] Val Loss=0.1627 | Acc=0.9310 | F1=0.9042
Saved FP32 checkpoint.

 PREPARING QAT MODEL 



/tmp/ipython-input-1611357868.py:39: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  quant.prepare_qat(self, inplace=True)
/usr/local/lib/python3.12/dist-packages/torch/ao/quantization/observer.py:246: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  war


TRAINING QAT MODEL



QAT Epoch 1: 100%|██████████| 1000/1000 [06:16<00:00,  2.65it/s, loss=0.0318]


[QAT] Val Loss=0.1796 | Acc=0.9290 | F1=0.9024

 QAT ARTIFACTS SAVED TO: /content/drive/MyDrive/ANLP_weights/qat
 Created ZIP file: /content/drive/MyDrive/ANLP_weights/qat.zip


('/content/drive/MyDrive/ANLP_weights/qat',
 '/content/drive/MyDrive/ANLP_weights/qat.zip')